<a href="https://colab.research.google.com/github/yeonghun00/dc/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# avoid ban
# http://www.useragentstring.com/
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

In [2]:
# crawling page
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [3]:
def page_view(gall_num, proxy_idx):
  try:
    url = 'https://gall.dcinside.com/board/view/?id=neostock&no=' + str(gall_num)
    result = requests.get(url, headers = headers, proxies={'https': proxies[proxy_idx]}, timeout=5)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    gall_num = gall_num
    title = bs_obj.find_all('span', {'class': 'title_subject'})[0].text
    content = bs_obj.find_all('div', {'class': 'write_div'})[0].text

    good = int(bs_obj.find_all('p', {'class': 'up_num'})[0].text)
    bad = int(bs_obj.find_all('p', {'class': 'down_num'})[0].text)

    date = bs_obj.find_all('span', {'class': 'gall_date'})[0].text
    view = bs_obj.find_all('span', {'class': 'gall_count'})[0].text
    comment = bs_obj.find_all('span', {'class': 'gall_comment'})[0].text

    date = datetime.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
    view = int(''.join(list(filter(str.isdigit, view))))
    comment = int(''.join(list(filter(str.isdigit, comment))))

    return [gall_num, date, title, content, good, bad, view, comment]
  except:
    pass

In [4]:
res = requests.get('https://free-proxy-list.net')
content = BeautifulSoup(res.text, 'lxml')
table = content.find('table')
rows = table.find_all('tr')
cols = [[col.text for col in row.find_all('td')] for row in rows]

proxies = []
proxy_index = 0

for col in cols:
    try:
        if col[4] == 'elite proxy' and col[6] == 'yes':
            proxies.append('https://' + col[0] + ':' + col[1])
    except:
        pass

from multiprocessing import Pool

with Pool(500) as p:
    li = p.starmap(page_view, list(zip([992009]*len(proxies), range(len(proxies)))))

working_idx = [x for x in range(len(proxies)) if li[x] != None]

In [5]:
working_idx

[4, 13, 18, 22, 29, 37, 40, 51]

In [12]:
idx_cycle = cycle(working_idx)
proxy_idx = next(idx_cycle)

def page_test(gall_num):
  url = 'https://gall.dcinside.com/board/view/?id=neostock&no=' + str(gall_num)
  result = requests.get(url, headers = headers, proxies={'https': proxies[proxy_idx]}, timeout=5)
  bs_obj = BeautifulSoup(result.content, "html.parser")

  gall_num = gall_num
  title = bs_obj.find_all('span', {'class': 'title_subject'})[0].text
  content = bs_obj.find_all('div', {'class': 'write_div'})[0].text

  good = int(bs_obj.find_all('p', {'class': 'up_num'})[0].text)
  bad = int(bs_obj.find_all('p', {'class': 'down_num'})[0].text)

  date = bs_obj.find_all('span', {'class': 'gall_date'})[0].text
  view = bs_obj.find_all('span', {'class': 'gall_count'})[0].text
  comment = bs_obj.find_all('span', {'class': 'gall_comment'})[0].text

  date = datetime.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
  view = int(''.join(list(filter(str.isdigit, view))))
  comment = int(''.join(list(filter(str.isdigit, comment))))

  result = [gall_num, date, title, content, good, bad, view, comment]

In [14]:
page_test(992006)

ConnectTimeout: ignored

In [6]:
from itertools import cycle

idx_cycle = cycle(working_idx)

def page_view2(gall_num):
  result = None
  while type(result) != list:
    proxy_idx = next(idx_cycle)
    try:
      url = 'https://gall.dcinside.com/board/view/?id=neostock&no=' + str(gall_num)
      result = requests.get(url, headers = headers, proxies={'https': proxies[proxy_idx]}, timeout=5)
      bs_obj = BeautifulSoup(result.content, "html.parser")

      gall_num = gall_num
      title = bs_obj.find_all('span', {'class': 'title_subject'})[0].text
      content = bs_obj.find_all('div', {'class': 'write_div'})[0].text

      good = int(bs_obj.find_all('p', {'class': 'up_num'})[0].text)
      bad = int(bs_obj.find_all('p', {'class': 'down_num'})[0].text)

      date = bs_obj.find_all('span', {'class': 'gall_date'})[0].text
      view = bs_obj.find_all('span', {'class': 'gall_count'})[0].text
      comment = bs_obj.find_all('span', {'class': 'gall_comment'})[0].text

      date = datetime.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
      view = int(''.join(list(filter(str.isdigit, view))))
      comment = int(''.join(list(filter(str.isdigit, comment))))

      result = [gall_num, date, title, content, good, bad, view, comment]
    except:
      return [None]
      
  return result

In [7]:
from multiprocessing import Pool

# start_page > end_page
start_page = 992855
end_page = 992000

with Pool(200) as p:
    galls = p.map(page_view2, range(end_page, start_page))

In [10]:
galls = [x for x in galls if x != None]

In [8]:
galls

[[None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [992007,
  datetime.datetime(2021, 5, 12, 13, 43, 42),
  '공매도 덕분에 여러 가정 구했네 ㅋㅋ',
  '\n암 이게 정상이지 ㅋㅋ ',
  0,
  0,
  107,
  0],
 [992008,
  datetime.datetime(2021, 5, 12, 13, 43, 43),
  '지존 괴롭다;',
  '\n매수버튼 왜 눌렀지; ',
  0,
  0,
  69,
  0],
 [None],
 [None],
 [992011,
  datetime.datetime(2021, 5, 12, 13, 44, 1),
  '하이닉스 131000원 300주인데',
  '\n자살은 어디서 하냐? ',
  0,
  0,
  137,
  1],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [992023,
  datetime.datetime(2021, 5, 12, 13, 44, 39),
  '진짜 공포면 갤질못한다',
  '\n손다리가 후들거리고 메스껍다지금 멀쩡하잖아 ',
  0,
  0,
  51,
  0],
 [None],
 [None],
 [None],
 [992027,
  datetime.datetime(2021, 5, 12, 13, 44, 54),
  '동국제강 갑자기 쏜다 ㅅㅅㅅㅅㅅㅅㅅ',
  '\n얼른타라\xa0 ',
  0,
  0,
  145,
  1],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [No

In [13]:
gall_dict = {}

for gall in galls:
  gall_id, date, title, content, good, bad, view, comment = gall
  gall_dict[gall_id] = {'date': date, 'title': title, 'content': content, 'good': good, 'bad': bad, 'view': view, 'comment': comment}

In [14]:
import pandas as pd
gall_df = pd.DataFrame.from_dict(gall_dict).transpose()
gall_df

,date,title,content,good,bad,view,comment
992845,2021-05-12 14:47:03,삼전 86층,\n살려줘라 ..,0,0,96,0
992846,2021-05-12 14:47:07,오늘 머공황 맞냐?,\n돈 꼴은 병신새끼 없제? 컄ㅋㅋㅋ,0,0,104,5
992847,2021-05-12 14:47:07,삼전 8만원에 사면 평타침?,\n어디까지 추락할까??,0,0,316,2
992848,2021-05-12 14:47:10,아니 한화생명 토스에서 주던거 존나 올랐네,\n근데 1주밖에 없음 ㅋㅋ ㅠㅠ3주 더 사볼까 완전 떡상하네..,0,0,43,0
992849,2021-05-12 14:47:15,이래도 철강을 안산다고?,\n세아베 미쳐따리,0,0,129,0
992850,2021-05-12 14:47:19,미체결이나 체결취소시 수수료안나가나요,\n2일차 주린이인데 너무 궁금해요 ㅠㅠ100만원대라 수수료가 크진않겟지만미체결이나...,0,0,104,2
992851,2021-05-12 14:47:29,동국 포강 세아베 다담아놨다,\n화이팅 누구든이겨라,0,0,64,0
992852,2021-05-12 14:47:39,동국아 이 씨발새끼야,\n닥치고 걸어,1,0,191,0
992853,2021-05-12 14:47:40,동국 아까 좀 더살걸,\n아깝네,0,0,165,2
992854,2021-05-12 14:48:07,삼전 부회장 한달전인가 8만2천에서 10억치 사지않았노? ㅋㅋㅋㅋㅋㅋㅋㅋ,\n물렸노?,1,0,231,3


In [15]:
title_words = gall_df['title'].values.tolist()
content_words = gall_df['content'].values.tolist()

In [ ]:
'''from google.colab import files

gall_df.to_csv('data.csv', encoding='utf-8-sig')
files.download('data.csv')'''

In [ ]:
def freq_dict(li):
  flat_li = [j for sub in list(map(str.split, li))  for j in sub]
  dic = {i:flat_li.count(i) for i in set(flat_li)}
  sorted_dic = sorted(dic.items(), key=lambda kv: kv[1], reverse=True)
  return sorted_dic

In [ ]:
freq_dict(title_words)

In [ ]:
freq_dict(content_words)